In [1]:
# default_exp utils 

# utils 

> General useful utility functions. 

In [2]:
#hide
from nbdev import *
from nbdev.showdoc import *

In [3]:
%nbdev_export
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from scipy import signal
import gym
from gym import wrappers
import math
import scipy
import matplotlib.pyplot as plt
from matplotlib import animation
from typing import Optional
from pathlib import Path
import time
import pickle as pkl
import os

color2num = dict(
    gray=30,
    red=31,
    green=32,
    yellow=33,
    blue=34,
    magenta=35,
    cyan=36,
    white=37,
    crimson=38,
)

In [4]:
%nbdev_export
def colorize(
    string: str,
    color: int,
    bold: Optional[bool] = False,
    highlight: Optional[bool] = False,
):
    """
    Colorize a string.
    This function was originally written by John Schulman.
    """
    attr = []
    num = color2num[color]
    if highlight:
        num += 10
    attr.append(str(num))
    if bold:
        attr.append("1")
    return "\x1b[%sm%s\x1b[0m" % (";".join(attr), string)

In [5]:
show_doc(colorize)

<h4 id="colorize" class="doc_header"><code>colorize</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>colorize</code>(**`string`**:`str`, **`color`**:`int`, **`bold`**:`Optional`\[`bool`\]=*`False`*, **`highlight`**:`Optional`\[`bool`\]=*`False`*)

Colorize a string.
This function was originally written by John Schulman.

In [6]:
%nbdev_export
def calc_logstd_anneal(n_anneal_cycles: int, anneal_start: float, anneal_end: float, epochs: int) -> np.ndarray:
    """
    Calculate log standard deviation annealing schedule. Can be used in PG algorithms on continuous action spaces.

    Args:
    - n_anneal_cycles (int): How many times to cycle from anneal_start to anneal_end over the training epochs.
    - anneal_start (float): Starting log standard deviation value.
    - anneal_end (float): Ending log standard deviation value.
    - epochs (int): Number of training cycles.

    Returns:
    - logstds (NumPy array): Log standard deviations for each training epoch.
    """
    if n_anneal_cycles > 0:
        logstds = np.linspace(anneal_start, anneal_end, num=epochs // n_anneal_cycles)
        for _ in range(n_anneal_cycles):
            logstds = np.hstack((logstds, logstds))
    else:
        logstds = np.linspace(anneal_start, anneal_end, num=epochs)

    return logstds

In [7]:
show_doc(calc_logstd_anneal)

<h4 id="calc_logstd_anneal" class="doc_header"><code>calc_logstd_anneal</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>calc_logstd_anneal</code>(**`n_anneal_cycles`**:`int`, **`anneal_start`**:`float`, **`anneal_end`**:`float`, **`epochs`**:`int`)

Calculate log standard deviation annealing schedule. Can be used in PG algorithms on continuous action spaces.

Args:
- n_anneal_cycles (int): How many times to cycle from anneal_start to anneal_end over the training epochs.
- anneal_start (float): Starting log standard deviation value.
- anneal_end (float): Ending log standard deviation value.
- epochs (int): Number of training cycles.

Returns:
- logstds (NumPy array): Log standard deviations for each training epoch.

In [8]:
%nbdev_export
def save_frames_as_gif(frames, filename=None):
    """
    Save a list of frames as a gif

    This code from this floydhub blog post: https://blog.floydhub.com/spinning-up-with-deep-reinforcement-learning/
    """
    # patch = plt.imshow(frames[0])
    fig = plt.figure()
    plt.axis("off")

    def animate(i):
        patch.set_data(frames[i])

    # anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
    anim = animation.ArtistAnimation(fig, frames, interval=50)
    if filename:
        anim.save(filename, writer="imagemagick")

In [9]:
show_doc(save_frames_as_gif)

<h4 id="save_frames_as_gif" class="doc_header"><code>save_frames_as_gif</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>save_frames_as_gif</code>(**`frames`**, **`filename`**=*`None`*)

Save a list of frames as a gif

This code from this floydhub blog post: https://blog.floydhub.com/spinning-up-with-deep-reinforcement-learning/

In [10]:
%nbdev_export
def _discount_cumsum(x: np.array, discount: float):
    """
    magic from rllab for computing discounted cumulative sums of vectors.

    input:
        vector x,
        [x0,
        x1,
        x2]

    output:
        [x0 + discount * x1 + discount^2 * x2,
        x1 + discount * x2,
        x2]
    """
    return scipy.signal.lfilter([1], [1, float(-discount)], x[::-1], axis=0)[::-1]

In [11]:
%nbdev_export
def conv2d_output_size(kernel_size, stride, sidesize):
    """Calculate output size of Conv2D layer given kernel_size, stride, and size of one side of input image. Assumes square input image."""
    return (sidesize - (kernel_size - 1) - 1) // stride + 1

def num2tuple(num):
    """Convert an input number to a tuple of (num, num)."""
    return num if isinstance(num, tuple) else (num, num)

def conv2d_output_shape(
    h_w: tuple,
    kernel_size: Optional[int] = 1,
    stride: Optional[int] = 1,
    pad: Optional[int] = 0,
    dilation: Optional[int] = 1
    ):
    """
    Calculate output shape of a Conv2D layer.

    Args:
    - h_w (tuple): tuple of input height and width
    - kernel_size (int): size of kernel
    - stride (int): kernel stride
    - pad (int): padding
    - dilation (int): dilation applied

    Returns:
    - h (int): height of output
    - w (int): width of output
    """
    h_w, kernel_size, stride, pad, dilation = num2tuple(h_w), \
        num2tuple(kernel_size), num2tuple(stride), num2tuple(pad), num2tuple(dilation)
    pad = num2tuple(pad[0]), num2tuple(pad[1])

    h = math.floor((h_w[0] + sum(pad[0]) - dilation[0]*(kernel_size[0]-1) - 1) / stride[0] + 1)
    w = math.floor((h_w[1] + sum(pad[1]) - dilation[1]*(kernel_size[1]-1) - 1) / stride[1] + 1)

    return h, w

def convtransp2d_output_shape(
    h_w: tuple,
    kernel_size: Optional[int] = 1,
    stride: Optional[int] = 1,
    pad: Optional[int] = 0,
    dilation: Optional[int] = 1,
    out_pad: Optional[int] = 0
    ):
    """
    Calculate output shape of a ConvTranspose2D layer.

    Args:
    - h_w (tuple): tuple of input height and width
    - kernel_size (int): size of kernel
    - stride (int): kernel stride
    - pad (int): padding
    - dilation (int): dilation applied
    - out_pad (int): output padding

    Returns:
    - h (int): height of output
    - w (int): width of output
    """
    h_w, kernel_size, stride, pad, dilation, out_pad = num2tuple(h_w), \
        num2tuple(kernel_size), num2tuple(stride), num2tuple(pad), num2tuple(dilation), num2tuple(out_pad)
    pad = num2tuple(pad[0]), num2tuple(pad[1])

    h = (h_w[0] - 1)*stride[0] - sum(pad[0]) + dialation[0]*(kernel_size[0]-1) + out_pad[0] + 1
    w = (h_w[1] - 1)*stride[1] - sum(pad[1]) + dialation[1]*(kernel_size[1]-1) + out_pad[1] + 1

    return h, w

In [12]:
%nbdev_export
class Saver:
    r"""
    A class to pickle generic Python objects saved over a model's training cycle.

    Args:
    - out_dir (str): Directory to save to.
    - keys (list): Keys to initialize the save dictionary with.
    """
    def __init__(self, out_dir: str, keys: Optional[list] = []):
        self.out_path = Path(out_dir)
        os.makedirs(self.out_path, exist_ok=True)
        self.saver_dict = {k: [] for k in keys} if len(keys) > 0 else {}

    def store(self, **kwargs):
        """Store input kwargs in save dictionary."""
        for k, v in kwargs.items():
            if k not in self.saver_dict.keys():
                self.saver_dict[k] = [v]
            else:
                self.saver_dict[k].append(v)

    def save(self):
        """Write save dictionary to .pkl file."""
        ct = time.time()
        if len(self.saver_dict) > 0:
            pkl.dump(
                self.saver_dict,
                open(
                    self.out_path / f"saver_out_{ct}.pkl", "wb"
                ),
            )

In [15]:
#hide
notebook2script()

Converted 00_utils.ipynb.
Converted 01_datasets.ipynb.
Converted 02_buffers.ipynb.
Converted 03_neuralnets.ipynb.
Converted index.ipynb.
